# A gradio webapp with Azure ChatGPT

<img src="bot.png">

In [1]:
import openai
import os
import gradio as gr
import openai
import random
import time
import sys

# from dotenv import load_dotenv

In [2]:
sys.version

'3.11.5 (main, Sep 14 2023, 09:05:47) [GCC 10.2.1 20210110]'

Loading configuration from GH Secrets

In [3]:
#load_dotenv("azure.env")

openai.api_type: str = "azure"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
# openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_version = os.getenv("AZURE_OPENAI_MODEL_CHAT_VERSION")

In [4]:
# deployment_id = "gpt-35-turbo-16k"
deployment_id = os.getenv("AZURE_OPENAI_MODEL_CHAT") # the GH secret contains you gpt deployment name

In [10]:
system_message = {"role": "system", "content": "You are an AI assistant."}

# https://huggingface.co/spaces/gradio/theme-gallery

with gr.Blocks() as azure_openai_bot:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    state = gr.State([])

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history, messages_history):
        user_message = history[-1][0]
        bot_message, messages_history = ask_gpt(user_message, messages_history)
        messages_history += [{"role": "assistant", "content": bot_message}]
        history[-1][1] = bot_message
        time.sleep(1)
        return history, messages_history

    # the definition that makes the call to the OpenAI ChatCompletion API sending the user message + history
    def ask_gpt(message, messages_history):
        messages_history += [{"role": "user", "content": message}]
        response = openai.ChatCompletion.create(
            engine=deployment_id,
            messages=messages_history
        )
        print(response["choices"][0]["message"]["content"], messages_history)
        return response["choices"][0]["message"]["content"], messages_history

    def init_history(messages_history):
        messages_history = []
        messages_history += [system_message]
        return messages_history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, state], [chatbot, state]
    )

    clear.click(lambda: None, None, chatbot, queue=False).success(
        init_history, [state], [state]
    )

If execute in GitHub Codespace, a pop-up window will propose to open the website. You can also open it clicking in **Ports** next to Terminal tab in the codespace, look for port 7860 and **Open in Browser**.

In [11]:
azure_openai_bot.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f98f40f2c5b10c8a62.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


I'm sorry, I don't understand your request. Could you please provide more information or a question you'd like me to assist you with? [{'role': 'system', 'content': 'You are an AI assistant.'}, {'role': 'user', 'content': 'a'}]
Hello! How can I assist you today? [{'role': 'system', 'content': 'You are an AI assistant.'}, {'role': 'user', 'content': 'a'}, {'role': 'assistant', 'content': "I'm sorry, I don't understand your request. Could you please provide more information or a question you'd like me to assist you with?"}, {'role': 'user', 'content': 'hello'}]
As an AI assistant, I can perform various tasks such as:

1. Setting reminders and alarms
2. Creating to-do lists
3. Providing weather updates
4. Answering general knowledge and trivia questions
5. Providing news updates
6. Making calculations and conversions
7. Setting and managing your schedule
8. Sending emails and messages
9. Providing navigation and traffic updates
10. Help with language translation

Is there anything specifi

In [12]:
azure_openai_bot.close()

Closing server running on port: 7860
